In [15]:
import pyspark.sql.functions as sf

# User Defined Functions

From time to time you hit a wall where you need a simple transformation, but Spark does not offer an appropriate function in the `pyspark.sql.functions` module. Fortunately you can simply define new functions, so called *user defined functions* or short *UDFs*.

# 1. Define Python Function

The first step probably always is to define a simple Python function which provides the required logic. In our case we want to use the existing function `html.escape` and check that it does what we need. In other cases you'd probably create your own python function

In [ ]:
import html

html.escape("Thelma & Louise")

# 2. Create UDF from Python Function

Now that we have an appropriate Python function, we need to *wrap* it and convert it to a Spark function. Spark functions do not work directly with values, but they expect *columns* (i.e. placeholders for values). This conversion from Python to Spark can easily be done using the Spark function `udf`.

In [7]:
import html

html_encode = sf.udf(lambda s: html.escape(s), StringType())

Create a small test data frame

In [8]:
df = spark.createDataFrame([('Alice & Bob',12),('Thelma & Louise',17)],['name','age'])
df.toPandas()

,name,age
0,Alice & Bob,12
1,Thelma & Louise,17


Use the UDF within a Spark `select` operation

In [9]:
result = df.select(html_encode('name').alias('html_name'))
result.toPandas()

,html_name
0,Alice &amp; Bob
1,Thelma &amp; Louise


As an alternative, you can also use a Python decorator for declaring a UDF:

In [11]:
@sf.udf(StringType())
def html_encode(s):
    return html.escape(s)

result = df.select(html_encode('name').alias('html_name'))
result.toPandas()

,html_name
0,Alice &amp; Bob
1,Thelma &amp; Louise


# 3. Complex return types

PySpark also supports complex return types, for example structs (or also arrays)

In [12]:
@sf.udf(StructType([
    StructField("org_name", StringType()), 
    StructField("html_name", StringType())
]))
def html_encode(s):
    return (s,html.escape(s))

result = df.select(html_encode('name').alias('both_names'))
result.toPandas()

,both_names
0,"(Alice & Bob, Alice &amp; Bob)"
1,"(Thelma & Louise, Thelma &amp; Louise)"


# 4. SQL Support

If you wanto to use the Python UDF inside a SQL query, you also need to register it, so PySpark knows its name.

In [13]:
html_encode = spark.udf.register("html_encode", lambda s: html.escape(s), StringType())

df.createOrReplaceTempView("famous_pairs")
result = spark.sql("SELECT html_encode(name) FROM famous_pairs")
result.toPandas()

,html_encode(name)
0,Alice &amp; Bob
1,Thelma &amp; Louise
